# Task Model Development

## Problem Defintion

We are facing a challenge in locating golf balls in an image taken over a golf course, which is meant to mimic a real golf course. The program should be able to provide the locations of the balls based on the input of a new image. We have a dataset consisting of 100 different images, each containing up to 14 golf balls. These images have been processed and bounding boxes have been added to indicate where the golf balls are located in the images. So our datasets contains raw images: "images" and labelled images with bounding boxes: "json_labeling".

We aim to develop a model that can receive an image of golf balls and return the coordinates of their locations.

You may use any third-party library of your choice.
Understanding of the code is essential.
If the model you use is not covered in this course, then we won’t evaluate you for 
a compre-hensive grasp of the theories behind it.  We would however ask you questions in 
a rather highlevel regarding the model you use.
You can build one or more models for your task

## Data Extraction & Data Processing



In [2]:
import os
import shutil # For copying files
import json 
from sklearn.model_selection import train_test_split # For splitting data

# Define the paths to your images and JSON annotations
images_path = '../images'
annotations_path = '../json_labeling'

# Load all image and annotation file names
images = [f for f in os.listdir(images_path) if f.endswith('.png')]
annotations = [f for f in os.listdir(annotations_path) if f.endswith('.json')]

# Ensure the images and annotations lists are sorted to maintain alignment
images.sort()
annotations.sort()

# Split data into train, validation, and test sets
train_images, test_images, train_annotations, test_annotations = train_test_split(images, annotations, test_size=0.30, random_state=42)
val_images, test_images, val_annotations, test_annotations = train_test_split(test_images, test_annotations, test_size=0.50, random_state=42)

# Helper function to copy files
def copy_files(files, source_dir, target_dir):
    for file in files:
        shutil.copy(os.path.join(source_dir, file), os.path.join(target_dir, file))

# Create directories for the dataset splits if they don't already exist
for split in ['train', 'validation', 'test']:
    for kind in ['images', 'annotations']:
        os.makedirs(f'data/{split}/{kind}', exist_ok=True)

# Copy the files to the respective directories
copy_files(train_images, images_path, 'data/train/images')
copy_files(train_annotations, annotations_path, 'data/train/annotations')
copy_files(val_images, images_path, 'data/validation/images')
copy_files(val_annotations, annotations_path, 'data/validation/annotations')
copy_files(test_images, images_path, 'data/test/images')
copy_files(test_annotations, annotations_path, 'data/test/annotations')

print("Data successfully organized into train, validation, and test sets.")


Data successfully organized into train, validation, and test sets.


In [3]:
import tensorflow as tf
import os
import shutil
import json
from PIL import Image
import io
from sklearn.model_selection import train_test_split

def create_tf_example(image_path, annotation_path):
    # Load the image and get its dimensions
    try:
        with tf.io.gfile.GFile(image_path, 'rb') as fid:
            encoded_jpg = fid.read()
    except IOError:
        print(f"Error: Could not read image {image_path}. Skipping example.")
        return None
    
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    try:
        image = Image.open(encoded_jpg_io)
    except IOError:
        print(f"Error: Failed to open image {image_path}. Skipping example.")
        return None
    width, height = image.size

    # Load annotations
    try:
        with open(annotation_path, 'r') as f:
            annotation = json.load(f)
    except FileNotFoundError:
        print(f"Error: Annotation file {annotation_path} not found. Skipping example.")
        return None

    # Prepare the example
    xmins = []  # List of normalized left x coordinates in bounding box (1 per box)
    xmaxs = []  # List of normalized right x coordinates in bounding box (1 per box)
    ymins = []  # List of normalized top y coordinates in bounding box (1 per box)
    ymaxs = []  # List of normalized bottom y coordinates in bounding box (1 per box)
    classes_text = []  # List of string class name of bounding box (1 per box)
    classes = []  # List of integer class id of bounding box (1 per box, assuming 'golf ball' is 1)

    for obj in annotation['annotations']:
        xmins.append(obj['xmin'] / float(width))
        xmaxs.append(obj['xmax'] / float(width))
        ymins.append(obj['ymin'] / float(height))
        ymaxs.append(obj['ymax'] / float(height))
        classes_text.append(obj['label'].encode('utf8'))
        classes.append(1)  # Assumes the class ID for 'golf ball' is 1

    if not xmins:  # Checks if there are any bounding boxes
        print(f"No bounding boxes found in {annotation_path}. Skipping example.")
        return None

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': tf.train.Feature(int64_list=tf.train.Int64List(value=[height])),
        'image/width': tf.train.Feature(int64_list=tf.train.Int64List(value=[width])),
        'image/filename': tf.train.Feature(bytes_list=tf.train.BytesList(value=[os.path.basename(image_path).encode('utf8')])),
        'image/source_id': tf.train.Feature(bytes_list=tf.train.BytesList(value=[os.path.basename(image_path).encode('utf8')])),
        'image/encoded': tf.train.Feature(bytes_list=tf.train.BytesList(value=[encoded_jpg])),
        'image/format': tf.train.Feature(bytes_list=tf.train.BytesList(value=[b'jpeg' if image_path.lower().endswith('.jpeg') else b'png'])),
        'image/object/bbox/xmin': tf.train.Feature(float_list=tf.train.FloatList(value=xmins)),
        'image/object/bbox/xmax': tf.train.Feature(float_list=tf.train.FloatList(value=xmaxs)),
        'image/object/bbox/ymin': tf.train.Feature(float_list=tf.train.FloatList(value=ymins)),
        'image/object/bbox/ymax': tf.train.Feature(float_list=tf.train.FloatList(value=ymaxs)),
        'image/object/class/text': tf.train.Feature(bytes_list=tf.train.BytesList(value=classes_text)),
        'image/object/class/label': tf.train.Feature(int64_list=tf.train.Int64List(value=classes)),
    }))
    return tf_example

def create_tf_records(images, annotations, output_path):
    writer = tf.io.TFRecordWriter(output_path)
    for image_path, annotation_path in zip(images, annotations):
        tf_example = create_tf_example(image_path, annotation_path)
        if tf_example:
            writer.write(tf_example.SerializeToString())
    writer.close()

# Example of how to call create_tf_records for the training dataset
train_images = [os.path.join('data/train/images', name) for name in os.listdir('data/train/images')]
#train_annotations = [os.path.join('data/train/annotations', name.replace('.json', '.png')) for name in train_images]
train_annotations = [f"data/train/annotations/label_{i+1}.json" for i in range(len(train_images))]

create_tf_records(train_images, train_annotations, 'train.tfrecord')

# Repeat for validation and test datasets...


Error: Annotation file data/train/annotations\label_1.json not found. Skipping example.
Error: Annotation file data/train/annotations\label_2.json not found. Skipping example.


KeyError: 'annotations'

## Modeling

## Performance Evaluation

## Result Explanation